# Update total_results with 2048-ensemble EEMD/CEEMDAN

Reads `Data_final/total_results.csv`, recomputes SLR using new decompositions
from `Data_mid/*_{EEMD,CEEMDAN}_2048.parquet`, and writes
`Data_final/total_results_2048.csv`. Linear SLR is also recomputed from monthly data.

In [36]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.dates as mdates

DATA_DIR = Path('Data')
MID_DIR = Path('Data_mid')
FINAL_DIR = Path('Data_final')
SRC_CSV = FINAL_DIR / 'total_results.csv'
DST_CSV = FINAL_DIR / 'total_results_2048_0.6.csv'
TRIALS = 2048
EPSILON = 0.6


In [37]:
# Helpers
def load_monthly_txt(station: str) -> pd.DataFrame:
    p = DATA_DIR / f'Monthly_{station}.txt'
    rows = []
    with open(p, 'r', encoding='utf-8', errors='ignore') as f:
        for ln in f:
            ln = ln.strip()
            if not ln: continue
            parts = ln.split()
            try:
                y, mo, d, hh, mm = map(int, parts[:5])
                val = float(parts[-1])
            except Exception:
                continue
            rows.append((f'{y:04d}-{mo:02d}-{d:02d} {hh:02d}:{mm:02d}', val))
    df = pd.DataFrame(rows, columns=['date','value'])
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    return df

def compute_slope_from_series(dates: pd.Series, values: pd.Series) -> float:
    x = np.arange(len(values))
    y = np.asarray(values, dtype=float)
    
    if len(x) < 2:
        return np.nan
    
    # curve_fit 대신 polyfit 사용하지만 동일한 계산
    params = np.polyfit(x, y, deg=1)  # [slope, intercept]
    pred = np.poly1d(params)(x)
    
    # 이전 코드와 완전히 동일한 계산
    _slr = (pred[-1] - pred[0]) / len(pred) * 10 * 12
    
    return float(_slr)

def compute_slope_from_parquet(station: str, kind: str) -> float:
    # kind in {'EEMD','CEEMDAN'}
    p = MID_DIR / f'{station}_{kind}_{TRIALS}_{EPSILON}.parquet'
    if not p.exists():
        return np.nan
    try:
        df = pd.read_parquet(p)
    except Exception:
        return np.nan
    # Assume columns like IMF_0 ... IMF_n; take last IMF as trend
    imf_cols = [c for c in df.columns if str(c).startswith('IMF_')]
    if not imf_cols: return np.nan
    # Determine last by numeric order
    order = sorted(imf_cols, key=lambda c: int(str(c).split('_')[-1]))
    y = df[order[-1]].values
    # index is datetime in parquet we wrote; if not, fallback to range index
    if isinstance(df.index, pd.DatetimeIndex):
        dates = pd.to_datetime(df.index)
    else:
        # reconstruct from monthly length assuming same as Monthly_* file
        mdf = load_monthly_txt(station)
        dates = mdf['date'][:len(y)]
    return compute_slope_from_series(dates, y)

def compute_linear_slope_from_parquet_sum(station: str, kind: str) -> float:
    """
    Compute linear SLR from the reconstructed series (sum of all IMFs)
    stored in `<station>_<kind>_<TRIALS>.parquet`.
    kind in {'EEMD','CEEMDAN'}.
    """
    p = MID_DIR / f'{station}_{kind}_{TRIALS}_{EPSILON}.parquet'
    if not p.exists():
        return np.nan
    try:
        df = pd.read_parquet(p)
    except Exception:
        return np.nan
    imf_cols = [c for c in df.columns if str(c).startswith('IMF_')]
    if not imf_cols:
        return np.nan
    y = df[imf_cols].sum(axis=1).values
    if isinstance(df.index, pd.DatetimeIndex):
        dates = pd.to_datetime(df.index)
    else:
        mdf = load_monthly_txt(station)
        dates = mdf['date'][:len(y)]
    return compute_slope_from_series(dates, y)


In [38]:
# Build fresh results from new 2048-ensemble outputs (no dependency on previous SLR results)
monthly_files = sorted([p for p in DATA_DIR.glob('Monthly_*.txt')])
stations = [p.stem.replace('Monthly_', '') for p in monthly_files]

# Optional metadata join: latitude/longitude from previous CSV if available
latlon_map = {}
if SRC_CSV.exists():
    prev = pd.read_csv(SRC_CSV)
    if 'station name' in prev.columns:
        prev = prev.rename(columns={'station name':'station'})
    if {'station','latitude','longitude'}.issubset(prev.columns):
        latlon_map = prev.set_index('station')[['latitude','longitude']].to_dict('index')

rows = []
for s in stations:
    m = load_monthly_txt(s)
    if m.empty:
        continue
    dstart = m['date'].iloc[0]
    dend = m['date'].iloc[-1]
    years = (dend - dstart).days / 365.0
    # Linear on original data
    slr_lin = compute_slope_from_series(m['date'], m['value'])
    # Last-IMF slopes from new 2048 parquets
    slr_eemd = compute_slope_from_parquet(s, 'EEMD')
    slr_ce = compute_slope_from_parquet(s, 'CEEMDAN')
    # Linear on reconstructed (sum of IMFs)
    slr_eemd_lin_sum = compute_linear_slope_from_parquet_sum(s, 'EEMD')
    slr_ce_lin_sum = compute_linear_slope_from_parquet_sum(s, 'CEEMDAN')
    lat = latlon_map.get(s, {}).get('latitude', np.nan)
    lon = latlon_map.get(s, {}).get('longitude', np.nan)
    rows.append({
        'station name': s,
        'latitude': lat,
        'longitude': lon,
        'date start': dstart.strftime('%Y-%m-%d'),
        'date end': dend.strftime('%Y-%m-%d'),
        'data length (year)': years,
        # Match original naming: the methodless column was CEEMDAN in prior results
        'yearly slr (mm/yr) CEEMDAN': slr_ce,
        'yearly slr (mm/yr) EEMD': slr_eemd,
        'yearly slr (mm/yr) Linear': slr_lin,
        # 'yearly slr (mm/yr) Linear_from_IMFs_EEMD_2048': slr_eemd_lin_sum,
        # 'yearly slr (mm/yr) Linear_from_IMFs_CEEMDAN_2048': slr_ce_lin_sum,
    })

out = pd.DataFrame(rows)
order = ['station name','latitude','longitude','date start','date end','data length (year)',
         'yearly slr (mm/yr) CEEMDAN','yearly slr (mm/yr) EEMD','yearly slr (mm/yr) Linear']
out = out[[c for c in order if c in out.columns]]
out.to_csv(DST_CSV, index=False)
DST_CSV


PosixPath('Data_final/total_results_2048_0.6.csv')

In [39]:
out

,station name,latitude,longitude,date start,date end,data length (year),yearly slr (mm/yr) CEEMDAN,yearly slr (mm/yr) EEMD,yearly slr (mm/yr) Linear
0,Anheung,36.673611,126.132222,1989-01-16,2020-12-16,31.936986,2.064728,1.876882,2.848369
1,Boryeong,36.406389,126.486111,1986-01-16,2020-12-16,34.939726,2.747323,2.593289,3.271571
2,Busan,35.096111,129.035556,1975-01-16,2020-12-16,45.947945,2.653208,2.322311,2.789885
3,Chujado,33.961667,126.300278,1986-01-16,2020-12-16,34.939726,2.449173,1.932183,2.937847
4,Gadeokdo,35.024722,128.810833,1983-01-16,2020-12-16,37.942466,2.197155,1.947798,2.648716
5,Geomundo,34.028333,127.309167,1985-01-16,2020-12-16,35.939726,1.950057,1.101613,2.909375
6,Gunsan,35.975556,126.563333,1983-01-16,2020-12-16,37.942466,2.515957,2.477159,3.208162
7,Heuksando,34.683889,125.436389,1982-01-16,2020-12-16,38.942466,2.555412,2.778307,3.040568
8,Incheon,37.451944,126.592222,1978-01-16,2020-12-16,42.945205,3.170525,2.811090,3.269977
9,Jeju,33.527500,126.543056,1978-01-16,2020-12-16,42.945205,3.104326,2.765519,3.502915


In [47]:
sorted_name = ['Incheon', 'Anheung', 'Boryeong', 'Gunsan', 'Wido', 'Mokpo', 'Heuksando', 'Chujado', 'Jeju', 'Seogwipo', 
               'Wando', 'Geomundo', 'Yeosu', 'Tongyeong', 'Gadeokdo', 'Busan', 'Ulsan', 'Pohang', 'Mukho', 'Sokcho', 'Ulleungdo']
out.set_index('station name').loc[sorted_name].reset_index()#.iloc[14:]['yearly slr (mm/yr) CEEMDAN'].mean()

,station name,latitude,longitude,date start,date end,data length (year),yearly slr (mm/yr) CEEMDAN,yearly slr (mm/yr) EEMD,yearly slr (mm/yr) Linear
0,Incheon,37.451944,126.592222,1978-01-16,2020-12-16,42.945205,3.170525,2.811090,3.269977
1,Anheung,36.673611,126.132222,1989-01-16,2020-12-16,31.936986,2.064728,1.876882,2.848369
2,Boryeong,36.406389,126.486111,1986-01-16,2020-12-16,34.939726,2.747323,2.593289,3.271571
3,Gunsan,35.975556,126.563333,1983-01-16,2020-12-16,37.942466,2.515957,2.477159,3.208162
4,Wido,35.618056,126.301944,1985-01-16,2020-12-16,35.939726,2.282205,2.148556,3.406989
5,Mokpo,34.779722,126.375556,1960-01-16,2020-12-16,60.958904,2.331025,1.936789,2.457058
6,Heuksando,34.683889,125.436389,1982-01-16,2020-12-16,38.942466,2.555412,2.778307,3.040568
7,Chujado,33.961667,126.300278,1986-01-16,2020-12-16,34.939726,2.449173,1.932183,2.937847
8,Jeju,33.527500,126.543056,1978-01-16,2020-12-16,42.945205,3.104326,2.765519,3.502915
9,Seogwipo,33.240000,126.561667,1985-01-16,2020-12-16,35.939726,2.374121,1.049513,3.265190


In [50]:
out.set_index('station name').loc[sorted_name].iloc[16:]['yearly slr (mm/yr) CEEMDAN'].mean()

2.7925034161994065

In [51]:
out.set_index('station name').loc[sorted_name].iloc[16:]['yearly slr (mm/yr) CEEMDAN'].std()

0.742193804578751

In [52]:
out.set_index('station name').loc[sorted_name].iloc[:16]['yearly slr (mm/yr) CEEMDAN'].mean()

2.4033560607651747

In [53]:
out.set_index('station name').loc[sorted_name].iloc[16:]['yearly slr (mm/yr) CEEMDAN'].std()

0.742193804578751

In [54]:
out.set_index('station name').loc[sorted_name].iloc[:]['yearly slr (mm/yr) CEEMDAN'].mean()

2.4960101930114207

In [55]:
x = out.set_index('station name').loc[sorted_name].iloc[:]['yearly slr (mm/yr) CEEMDAN']

In [56]:
x.min(), x.max()

(1.8672749233310189, 4.069620657540176)